# VPC Components  
A *VPC (Virtual Private Cloud)* is a secure, isolated cloud environment hosted within a public cloud. A private cloud is available only to one organisation, whereas a public cloud is shared cloud infrastructure, where multiple customers can use the same machines for different purposes. In this notebook we will outline and describe the landscape of components used for building and operating a VPC.

### Key Features & Benefits of a VPC
- Full and comprehensive control over the virtual networking environment of a group of resources
- Allows for the division of a large network into smaller parts
- Allows for granular control of the flow of traffic in and out of specific CIDR ranges, including automatically determining where that traffic should go depending on it's volume
- Sophisticated network monitoring and logging capabilities 
- Easy to implement a hybrid cloud deployment by connecting the VPC to an on-premises infrastructure 

### Use Cases of a VPC
- **Hosting a public facing website:** When hosting a website you may need multiple compute instances combined with a database solution. A VPC allows you to group these resources together and then add layers of security and traffic monitoring so you can control the inbound and outbound traffic to your site to avoid attacks.
- **Connecting branch offices with a cloud system:** A VPC allows you to group resources together into their own networks and create rules for the inbound and outbound traffic. You could easily connect each of these smaller networks with the respective corporate branch on-premises system.
- **Hosting a multi-tier web application:** This use case requires more restrictions for communication between your different server layers, such as web servers, app servers, and database servers. A VPC is an ideal solution for such a scenario as it allows you to create private or public networks within a larger network, so your servers storing sensitive database information are completely inaccessible from the internet.

When creating a VPC, you define an IP address range using CIDR notation. AWS recommends that you use a private IPv4 address range for setting up your VPC, although it is possible to define a publicly routable CIDR block instead. Doing this would require you to own or have access to this public range of IP addresses, however. For the remainder of this notebook we will discuss VPCs as if they have been defined with a private IPv4 address range.

AWS automatically provides a *default VPC* in each AWS region. The default VPC simplifies the process of getting started with AWS resources by providing pre-configured settings and infrastructure. It includes a pre-defined CIDR block and a public subnets (we will cover this in the next section). However, you also have the option to create a *custom VPC*. A custom VPC allows for more control and customization, giving you the flexibility to define your own CIDR block and subnets.

## Subnets
*Subnets* are subdivisions of the IP address range defined for the VPC. They allow you to split the VPC into smaller sub-networks, or subnets. This is useful because it means you can control the networking around cloud resources by grouping them together in subnets. 

### Public vs Private Subnets
Before discussing *public* and *private subnets* it is important to remember the distinction between public and private IP addresses: subsets of the total range of IP addresses available are reserved to be private IP addresses, so they are not publicly accessible from the internet. Conversely, public IP addresses are assigned to devices so that they can interact with the internet. 

Building on this idea, subnets can be configured to be either public or private: accessible from the internet or not, depending on the use case of the resources inside each subnet.

For example: a private subnet would be used for resources such as database servers which need high levels of protection, and are only accessed by other resources in your VPC, so no public internet connection is required. Public subnets are used for resources which need to be accessed from the internet, such as public-facing web servers.

When configuring a subnet in an Amazon VPC, you can allow AWS to auto-assign public IP addresses to instances launched in that subnet. This does not mean those instances are then publicly accessible from the internet; the public IP address assigned to the instances is picked from a pool of protected, public IP addresses owned by Amazon. It is used by other AWS networking services such as an Internet Gateway.

>An *Internet Gateway (IGW)* is a horizontally scalable, highly available AWS-managed service that acts as a link between a VPC and the internet. It facilitates secure communication between instances in the VPC and external networks, such as the public internet or other AWS services.

A *public subnet* is defined by AWS as one which has a direct route to an Internet Gateway, and can therefore access the internet and be accessed from the internet. A *private subnet* is one which does not have a direct route to an Internet Gateway.

## NAT Services
The primary advantage of private subnets is security; instances in private subnets are not accessible from the internet because they dont have a direct route through an Internet Gateway. But without an Internet Gateway, instances in a private subnet can not send outbound traffic either, because they don't have a publicly routable IP address. In order to make this possible, we need to perform NAT.

>As a reminder, *NAT (Network Access Translation)* is the process of assigning public IP addresses to devices and instances so they can be accessed outside a private network such as a private subnet or LAN. Without NAT, resources in a private subnet cannot communicate with anything outside of that VPC. 

Traditionally, you could provision a special type of EC2 instance in a public subnet which would perform NAT and allow instances in a private subnet to send outbound traffic. This method is known as a *NAT Instance*. Although this is still possible, it less common today because it is difficult to scale and is unreliable: if that EC2 instance fails then your private subnet is cut off. As a solution to this, Amazon created NAT Gateways.

### NAT Gateways
A *NAT Gateway* is a managed AWS service which performs NAT on instances in private subnets so that they can connect to the internet. Each NAT Gateway is created in a specific Availability Zone, with redundancy within that Availability Zone. They can scale automatically and can handle large volumes of traffic. When you create a NAT Gateway, you specify one of the following connectivity types:

- **Public:** Choosing this option allows instances in private subnets to connect to the internet, but cannot receive unsolicited inbound connections. This method requires the NAT Gateway to be provisioned in a public subnet, because it works in conjunction with an Internet Gateway to allow outbound traffic from the private subnet to the internet. 
- **Private:** Private NAT Gateways only allow traffic from instances to other VPCs on your AWS account or a connected on-premises network. Any traffic sent from a private NAT Gateway through an Internet Gateway will be automatically dropped.

#### Elastic IP Addresses
An *elastic IP address* in AWS is a public IPv4 address which you can associate with an instance in your VPC, and if that instance fails the elastic IP address can quickly and automatically remap itself to another instance within the same VPC. This is useful because it means you can ensure very little downtime for an application or server, should an instance fail.

When you create a public NAT gateway you must associate an elastic IP address with the NAT gateway at creation. AWS does this automatically for you by default, but you can create and assign your own. You cannot associate an elastic IP address with a private NAT gateway.

## Route Table
A *route table* contains a set of rules, called routes, that determine where network traffic from your subnet is directed. When you create a VPC with AWS you get a route table by default, called the *main route table*, which you can add routes to, or you can create your own, *custom route table*.

The two important columns in a route table are the *destination* and *target*. The destination is the range of IP addresses where you want the traffic to go. The 
target is the connection through which the traffic should go, such as a NAT Gateway or Internet Gateway. 

For example, to allow traffic from your subnet to anywhere on the public internet, you would add the following route to your subnet route table:

Destination | Target
------------|--------
`0.0.0.0/0` |`internetgateway-id`

## Elastic Load Balancing
*Elastic load balancing* is a method of distributing traffic across different targets such as EC2 instances, containers, and IP addresses so that no one instance becomes overwhelmed. The load balancer serves as a single point of contact for incoming requests, and then distributes them across *targets* in *target groups* (instances grouped together to decide where traffic goes). The load balancer monitors the health of its registered targets and ensures that it routes traffic only to healthy targets.

You also create *listeners* to check for connection requests from clients, and *listener rules* to route requests from clients to the targets.

There are four different types of load balancer in AWS:

- **Network**: For TCP, UDP, and TLS traffic where extreme performance is required
- **Application**: For HTTP and HTTPS traffic. The load balancer routes based on the content of the request.
- **Gateway**: To deploy, scale, and manage virtual appliances such as firewalls
- **Classic**: Although still possible to create a classic load balancer, it is outdated compared to the other options because the intended use case was with a special type of EC2 called an *EC2-Classic*, which has now been deprecated.

[NOTE: this topic has turned out to be relatively large, as in it has lots of components to it. Considering potentally adding a hands-on, regardless of whether we include it in the project or not.]
    

## Hands-On: Creating a VPC with Public and Private Subnets
### Step 1: Create the VPC
- After logging in to the AWS console, navigate to VPC using the search bar and select *Create VPC*

- On the next screen, select *VPC only*. We are going to manually create the other resources.
- Enter a name for your VPC
- IPv4 CIDR block: Select *IPv4 CIDR manual input*
- Define your CIDR block. For the purposes of this Hands-On we will use `10.0.0.0/24`, which contains 256 IP addresses.
- IPv6 CIDR block: Select *No IPv6 CIDR block*
- Tenancy: Select *Default*. This attribute determines if instances that are launched into the VPC will run on hardware that's shared with other AWS accounts, or on dedicated for your use only.
- Press *Create VPC*

<p align="center"> <img src="images/1-vpc-settings.png" height="587" width="508"/> </p>

Once you have created your VPC you will be redirected back to the *Your VPCs* page. You should be able to see two VPCs in the list: the first one is the default VPC that comes with your AWS account, the second is the one we just created, with the IPv4 CIDR block of `10.0.0.0/24`.

### Step 2. Create Two Subnets
In this step we are going to create two subnets, which will be configured as public and private later

- From the main VPC dashboard, on the sidebar, click *Subnets*. There will be three subnets listed here already. These are for the default VPC, one subnet for each of the availability zones in your region.
- Press *Create Subnet*
- Select the VPC we created a moment ago
- Enter a name. This step is actually optional, but doing so allows you to more easily differentiate between subnets. Call this first subnet `my-test-public-subnet-1`, for example
- Choose an Availability Zone
- Enter `10.0.0.0/28` for the *IPv4 subnet CIDR block*. This subnet has 16 IP addresses.
- Leave everything else as it is

- Complete this step once again to create a second subnet, but calling it something like `my-test-private-subnet-1` instead. Make sure to select a different CIDR block so it doesn't overlap with the public subnet.

<p align="center"> <img src="images/2-subnet-settings.png" height="587" width="508"/> </p>

### Step 3: Configure public subnet
In this step we are going to allow the instances in this subnet to be auto-assigned a public IP address owned by Amazon.

- Select your public subnet

- Press *Actions* then *Edit subnet settings*
- Select the box for *Enable auto-assign public IPv4 address*

### Step 4: Create the Internet Gateway
In this step we are going to create an Internet Gateway which will allow the public subnet to communicate with the internet.

- From the main VPC dashboard, on the sidebar, press *Internet Gateways*

- There will be one there already, this is the default IGW. Click *Create internet gateway*.
- Enter a name. Press *Create Internet Gateway*.
- On the next screen, press *Actions*, then *Attach to a VPC*
- Select the VPC you created

### Step 5: Create the NAT Gateway
- From the main VPC dashboard, on the sidebar, press *NAT Gateways*, then *Create NAT Gateway*

- Enter a name for your NAT Gateway
- Select the private subnet you created
- For connectivity type, select *Public*
- Press *Allocate Elastic IP*
- Press *Create NAT gateway*

<p align="center"> <img src="images/5-create-nat-gateway.png" height="536" width="505"/> </p>

### Step 6: Create Route Tables
The last step of this process is to create custom route tables, and then associate them with the relevant subnet. The private subnet will be routed to the NAT Gateway you just created, and the public subnet will be routed to the Internet Gateway.

- From the main VPC dashboard, on the sidebar, press *Route Tables*

- Select *Create route table*
- Enter a name corresponding to one of the subnets (`public-route-table`, for example). Select your VPC, then press *Create route table*.
- From the next screen, press *Subnet associations*, then the *Edit subnet associations* button underneath *Explicit subnet associations* (see below screenshot)

<p align="center"> <img src="images/6-edit-subnet-associations.png" height="533" width="1083"/> </p>

- On the next screen, select the corresponding subnet, then press *Save assocations*

- Next, in the *Routes* pane of the main route table page, press *Edit routes*
- Press *Add route*, then enter `0.0.0.0/0` under *Destination* and select the Internet Gateway you created for *Target*. This will allow any traffic out of the subnet to go through the Internet Gateway and be able to commmunicate with the internet. Press *Save changes*.

- Repeat this whole step for the private subnet and route table, but make the NAT Gateway you have created the target in the route table instead of the Internet Gateway.

## Key Takeaways
- A VPC is a secure, isolated cloud environment within a public cloud. It allows for comprehensive control of the networking environment surrounding groups of resources, by defining a range of IP addresses which you have complete control over.

- Subnets are subdivisions of that IP address range, they can be made public or private
- An Internet Gateway is a service which acts as a link between a VPC and the internet. It is required in order for instances in subnets to connect to the internet, other VPCs or an on-premises network.
- A NAT Gateway is a managed service which allows a private subnet to send outbound traffic to the internet, other VPCs or a connected on-premises network. They can be configured with public or private connectivity: a public NAT Gateway creates a connection with the internet, a private NAT Gateway only allows communication with other VPCs on your account and a connected on-premises network.
- An elastic IP address is a public IPv4 address owned by Amazon which upon failure of an instance it's associated with, can automatically remap itself to another instance to avoid downtime
- A route table is a set of rules which determine where network traffic from your subnet should be directed
- An elastic load balancer is a service which provides a single point of contact in your VPC to then distribute traffic across different targets such as EC2 instances. There are three types of load balancer you need to know about: network, application and gateway.